In [23]:
import cv2
import numpy as np
from cv2 import VideoWriter, VideoWriter_fourcc

In [24]:
# Create a light gray image
frame = cv2.imread(filename='white_background.jpg')
# print(frame.shape)
overlay = cv2.imread(filename='car.png')
overlay = cv2.resize(overlay,(240,300))
# print(overlay.shape)
x_offset=500
y_offset=600
frame[y_offset:y_offset+overlay.shape[0], x_offset:x_offset+overlay.shape[1]] = overlay
cv2.imwrite("Generated_frame.png", frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [25]:
# Initialize video parameters
width = 1920
height = 1080
FPS = 2
seconds = 15

In [26]:
fourcc = VideoWriter_fourcc(*'MP42')
video = VideoWriter('./HUD.mp4', fourcc, float(FPS), (width, height))
for _ in range(FPS * seconds):
    frame = cv2.imread(filename='white_background.jpg')
    frame = cv2.resize(frame,(1920,1080))
    # print(frame.shape)
    overlay = cv2.imread(filename='car.png')
    overlay = cv2.resize(overlay,(240,300))
    # print(overlay.shape)
    y_offset -= 20
    frame[y_offset:y_offset+overlay.shape[0], x_offset:x_offset+overlay.shape[1]] = overlay
    video.write(frame)
video.release()